<h1>Fit a global CNN for all machines at the same time on the pngs files</h1>

In [ ]:
!pip install tensorflow

In [2]:
"""
    Fit a global CNN on all machines 
    from: https://keras.io/examples/vision/image_classification_from_scratch/
"""

import datetime
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# add parent folder to path 
from pathlib import Path
path = Path(os.getcwd())
str_parent = str(path.parent.absolute())
if not str_parent in sys.path:
    sys.path.append(str_parent)
import utils 
from SoundFile import SoundFile



In [4]:
use_folder = '../../_data_png_cnn2/machines/'
data_path = Path(use_folder)
folders_classes = [f for f in data_path.iterdir() if f.is_dir()]
# print('foldersClasses: ', folders_classes)
# for p in folders_classes:
#     print('p: ', p)
num_classes = len(folders_classes)
print('num_classes: ', num_classes)

# sys.exit()

image_size = (333, 216)
batch_size = 32
data_augmentation = keras.Sequential(
    [
        # layers.experimental.preprocessing.RandomFlip("horizontal"),
        # layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)


num_classes:  6


In [5]:
# Initialising the CNN v4: make model
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)
    
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256]: #, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

model = make_model(input_shape=image_size + (3,), num_classes=num_classes)
epochs = 8 # 50

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    # loss="categorical_crossentropy", # n classes - 2D shape (n_samples, n_class)
    loss="sparse_categorical_crossentropy", # n classes - 1D integer encoded target (n_class)
    metrics=["accuracy"],
)

In [6]:
# Found 17858 files belonging to 6 classes.
# Using 14287 files for training.
# Using 3571 files for validation.

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    use_folder,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    use_folder,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
model.fit(
    train_ds, epochs=epochs, validation_data=val_ds,
)



Found 19219 files belonging to 6 classes.
Using 15376 files for training.
Found 19219 files belonging to 6 classes.
Using 3843 files for validation.
Epoch 1/8
481/481 [==============================] - 2684s 6s/step - loss: 0.3258 - accuracy: 0.8714 - val_loss: 6.7060 - val_accuracy: 0.2027
Epoch 2/8
481/481 [==============================] - 2742s 6s/step - loss: 0.0940 - accuracy: 0.9674 - val_loss: 0.2394 - val_accuracy: 0.9334
Epoch 3/8
443/481 [==========================>...] - ETA: 3:31 - loss: 0.0569 - accuracy: 0.9816

In [ ]:
filename_classifier = '../../_classifier_cnn2/global_cnn.h5'
model.save(filename_classifier)

print('model saved: ', filename_classifier)
